In [ ]:
# suppress qiskit 1.0 deprecation warnings:
import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

from qiskit import QuantumCircuit, Aer, execute
import src.qcm_sim as sim
import src.compile_keys as keys

# import circuits:
import circuits.hidden_shift as hshift 

In [ ]:
m = 3
n = 2*m
c = int(m/3)

# Generate a random hidden shift
shift = [1 for _ in range(n)]

# create qiskit circuit
qc = hshift.hidden_shift(n,c,shift,is_clifford = False)
qc_qasm = qc.qasm()

print(qc)

In [ ]:
# Qiskit simulation
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()
counts = result.get_counts()

counts

In [ ]:
# number of samples:
number_of_t_gates = 2*7*c
epsilon = 0.1
prob_fail = 0.01
hoeffding_samples = keys.compute_hoeffding_samples(number_of_t_gates,epsilon,prob_fail)

hoeffding_samples

In [ ]:
# Shift string
print(f"\nHidden Shift: {shift}\n")

# QCM simulation
results = sim.run_qcm(qc_qasm,shots=hoeffding_samples)